## Testing efficient methods for evaluation

* Using prequential, test-then-train and windowed evaluation. We show how the *_fast versions can performance. 
* The *_fast methods rely on the MOA implementation ```PrequentialEvaluation(stream, learner, ...)``` from ```moa.evaluation.EfficientEvaluationLoops```
* Performance may be affected due to data reading and where data sits (i.e. in memory or in disk).
  * When working with CSV files, the conversion process involves using NumpyStream, resulting in data being stored in memory as an Instances object (ARFF). As a result, the *_fast evaluation methods do not yield any performance improvement.
  * When ARFF files are employed, the data resides on disk, and each call to ```next_instance()``` necessitates reading a new row from the file. This can be perceived as a potential bottleneck, and future implementations could explore ways to enhance data access efficiency (like caching).

* [EXPERIMENTAL] Changed the behavior of *_evaluation() and *_evaluation_fast() such that if NumpyStream is used, then defaults to *_evaluation() and if not, *_evaluation_fast() is used. **Setting optimise=False disables this behavior.**

**Notebook last updated on 08/12/2023**

# File paths

In [1]:
hyper_arff_file_path = '../data/Hyper100k.arff'
# Stream with 580k instances and around 100 features (csv)
covtfd_arff_file_path = '../data/covtFD.arff'
covtfd_csv_file_path = '../data/covtFD.csv'

## Real stream (arff) with 100k and ARF

```EvaluatePrequential -l (meta.AdaptiveRandomForest -l (ARFHoeffdingTree -k 6 -e 2000000 -g 50 -c 0.01) -s 30 -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01)) -s (ArffFileStream -f /Users/gomeshe/Desktop/data/Hyper100k.arff) -e BasicClassificationPerformanceEvaluator -f 1000000```

In [2]:
from capymoa.stream.stream import stream_from_file
from capymoa.evaluation import test_then_train_evaluation_fast
from capymoa.learner.classifier import AdaptiveRandomForest

learner = AdaptiveRandomForest(ensemble_size=30)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=None, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

capymoa_root: /home/antonlee/github.com/tachyonicClock/MOABridge/src/capymoa
MOA jar path location (config.ini): jar/moa.jar
JVM Location (system): 
JAVA_HOME: /usr/lib/jvm/java-17-openjdk
JVM args: ['-Xmx8g', '-Xss10M']
Sucessfully started the JVM and added MOA jar to the class path


{'learner': 'AdaptiveRandomForest',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f45dce580>,
 'wallclock': 39.120577573776245,
 'cpu_time': 51.25991}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


87.5


In [3]:
from capymoa.stream.stream import stream_from_file
from capymoa.evaluation import test_then_train_evaluation
from capymoa.learner.classifier import AdaptiveRandomForest

learner = AdaptiveRandomForest(ensemble_size=30)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=None, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'AdaptiveRandomForest',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f4010d8b0>,
 'wallclock': 42.73572397232056,
 'cpu_time': 47.420534}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


87.5


## Synthetic 10kk stream (RTG) with Naive Bayes

```EvaluatePrequential -l bayes.NaiveBayes -e BasicClassificationPerformanceEvaluator -i 10000000 -f 10000000```
* Using a synthetic generator with a massive amount of instances (e.g. 10 million) has a huge impact whether we use the fast or *python* version of the evaluation. It goes from 12s to 140s/150s.  

In [4]:
from capymoa.stream.stream import RandomTreeGenerator
from capymoa.learner import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation_fast
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())
stream = RandomTreeGenerator()

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=10000000, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f441ff610>,
 'wallclock': 6.884829044342041,
 'cpu_time': 7.420822999999999}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


73.64807


In [5]:
from capymoa.stream.stream import RandomTreeGenerator
from capymoa.learner.learners import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())
stream = RandomTreeGenerator()

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=10000000, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5eb4f0ac10>,
 'wallclock': 156.89642763137817,
 'cpu_time': 157.20156599999999}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


73.64807


## Real stream (covtFD) 580k+ instances with more than 100 features. 

* Using an arff file (which triggers ARFFFileReader internally) shows a 20% improvement in run time when using the fast version in comparison to the *python* one. It goes from 40s to 50s. 

In [6]:
from capymoa.stream.stream import stream_from_file
from capymoa.learner.learners import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation_fast
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

stream = stream_from_file(path_to_csv_or_arff=covtfd_arff_file_path, class_index=-1)

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=None, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f441ff610>,
 'wallclock': 34.795207500457764,
 'cpu_time': 35.897785999999996}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


52.24272862303812


In [7]:
from capymoa.stream.stream import stream_from_file
from capymoa.learner.learners import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

stream = stream_from_file(path_to_csv_or_arff=covtfd_arff_file_path, class_index=-1)

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=None, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f4011d040>,
 'wallclock': 48.09580206871033,
 'cpu_time': 48.080466}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


52.24272862303812


# Testing  ```prequential_evaluation_fast``` 
## SRP100 + real dataset with 100k instances

```EvaluatePrequential -l (meta.StreamingRandomPatches -s 100) -s (ArffFileStream -f ./data/RBFm/RBFm_100k.arff) -f 1000```
* When using a more demanding classifier (such as StreamingRandomPatches with 100 learners) with an ARFF file the impact of the evaluation function becomes negligiable (obtained about the same wallclock time of 200s).

In [8]:
from capymoa.stream import stream_from_file
from capymoa.learner import MOAClassifier
from capymoa.evaluation import prequential_evaluation_fast
from moa.classifiers.meta import StreamingRandomPatches

cl_SRP = MOAClassifier(moa_learner=StreamingRandomPatches(), CLI='-s 100', random_seed=1)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = prequential_evaluation_fast(stream=stream, learner=cl_SRP, max_instances=None, window_size=10000)

display(results)
display(results['windowed'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'StreamingRandomPatches',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f401174f0>,
 'windowed': <capymoa.evaluation.evaluation.ClassificationWindowedEvaluator at 0x7f5f40117700>,
 'wallclock': 154.49833822250366,
 'cpu_time': 178.15130800000003}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,10000.0,87.49,74.981221,75.039904,74.828974
1,20000.0,88.92,77.817008,78.107093,77.383139
2,30000.0,88.75,77.508214,77.468456,77.625298
3,40000.0,89.73,79.457096,79.731597,79.298529
4,50000.0,89.87,79.738788,80.172245,79.493927
5,60000.0,90.11,80.212371,80.032304,79.795710
6,70000.0,89.99,79.981108,79.931836,79.980000
7,80000.0,90.30,80.602637,80.727200,80.272524
8,90000.0,90.35,80.698068,80.330208,80.799841
9,100000.0,90.08,80.154084,80.227227,79.951496


89.559


In [9]:
from capymoa.stream import stream_from_file
from capymoa.learner import MOAClassifier
from capymoa.evaluation import prequential_evaluation
from moa.classifiers.meta import StreamingRandomPatches

cl_SRP = MOAClassifier(moa_learner=StreamingRandomPatches(), CLI='-s 100', random_seed=1)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = prequential_evaluation(stream=stream, learner=cl_SRP, max_instances=None, window_size=10000, optimise=False)

display(results)
display(results['windowed'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'StreamingRandomPatches',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f40117760>,
 'windowed': <capymoa.evaluation.evaluation.ClassificationWindowedEvaluator at 0x7f5f40117e50>,
 'wallclock': 160.10862493515015,
 'cpu_time': 180.73510299999998}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,10000.0,87.49,74.981221,75.039904,74.828974
1,20000.0,88.92,77.817008,78.107093,77.383139
2,30000.0,88.75,77.508214,77.468456,77.625298
3,40000.0,89.73,79.457096,79.731597,79.298529
4,50000.0,89.87,79.738788,80.172245,79.493927
5,60000.0,90.11,80.212371,80.032304,79.795710
6,70000.0,89.99,79.981108,79.931836,79.980000
7,80000.0,90.30,80.602637,80.727200,80.272524
8,90000.0,90.35,80.698068,80.330208,80.799841
9,100000.0,90.08,80.154084,80.227227,79.951496


89.559


# Testing with a large CSV file

* We can observe that there is not much difference between the evaluation functions when we are using the CSV which is eventually loaded to memory.
* Loading the CSV to memory may take a while if it is a massive dataset (this one has about 600k instances and more than 100 features). 

In [10]:
%%time
from capymoa.stream import stream_from_file

# Loads the csv data to memory, it should take from 1 minute to 2 minutes. 
stream = stream_from_file(path_to_csv_or_arff=covtfd_csv_file_path, class_index=-1)

CPU times: user 3min 58s, sys: 2.63 s, total: 4min 1s
Wall time: 2min 9s


In [11]:
from capymoa.learner import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=None, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f40117a90>,
 'wallclock': 20.370754718780518,
 'cpu_time': 20.368808999999942}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866


In [12]:
from capymoa.learner import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=None, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5e6ed8a460>,
 'wallclock': 20.54651188850403,
 'cpu_time': 21.560857000000055}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866


## Using prequential_evaluation_fast

In [13]:
from capymoa.learner import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = prequential_evaluation_fast(stream=stream, learner=learner, max_instances=None, window_size=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5f40117490>,
 'windowed': None,
 'wallclock': 20.864585399627686,
 'cpu_time': 20.857773000000066}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866


In [14]:
from capymoa.learner import MOAClassifier
from capymoa.evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = prequential_evaluation(stream=stream, learner=learner, max_instances=None, window_size=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <capymoa.evaluation.evaluation.ClassificationEvaluator at 0x7f5e6ed8a100>,
 'windowed': None,
 'wallclock': 20.503072023391724,
 'cpu_time': 20.497254999999996}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866
